In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import talib
from ta.momentum import RSIIndicator
from ta.trend import MACD
from ta.momentum import StochasticOscillator
from ta.trend import CCIIndicator
from ta.momentum import ROCIndicator
from ta.momentum import RSIIndicator
from ta.volatility import AverageTrueRange
from scipy.optimize import brute
from datetime import datetime, timedelta
from backtrader import Cerebro, TimeFrame
import backtrader.analyzers as btanalyzers
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings('ignore')

In [4]:
df = pd.read_csv(f"../data/BTCUSDT_1min_2024-05-01_to_2025-05-01.csv", parse_dates=["timestamp"])
df

,timestamp,open,high,low,close,volume
0,2024-04-30 16:00:00,60796.86,60819.07,60713.22,60815.04,55.38665
1,2024-04-30 16:01:00,60815.04,60815.05,60688.80,60707.61,26.68472
2,2024-04-30 16:02:00,60707.61,60707.61,60616.04,60671.50,41.16678
3,2024-04-30 16:03:00,60671.51,60824.38,60649.01,60794.13,31.65158
4,2024-04-30 16:04:00,60794.12,60892.41,60793.97,60892.40,29.64754
...,...,...,...,...,...,...
525995,2025-04-30 22:35:00,94280.00,94311.75,94251.22,94265.38,12.98149
525996,2025-04-30 22:36:00,94265.39,94284.63,94264.52,94284.62,3.23475
525997,2025-04-30 22:37:00,94284.62,94293.24,94277.72,94284.61,8.49435
525998,2025-04-30 22:38:00,94284.61,94284.61,94268.00,94271.31,6.88779


### 1. RSI

In [5]:
# RSI动量
def rsi_momentum(close, window=14, overbought=70, oversold=30):
    rsi = talib.RSI(close, window)
    return np.where(rsi > overbought, -1, np.where(rsi < oversold, 1, 0))

# MACD动量
def macd_momentum(close, fast=12, slow=26, signal=9):
    macd, signal, _ = talib.MACD(close, fast, slow, signal)
    return np.where(macd > signal, 1, -1)

# CCI动量
def cci_momentum(high, low, close, window=20, threshold=100):
    cci = talib.CCI(high, low, close, window)
    return np.where(cci > threshold, -1, np.where(cci < -threshold, 1, 0))

# 随机振荡器(Stochastic Oscillator)
def stochastic_momentum(high, low, close, k_window=14, d_window=3):
    slowk, slowd = talib.STOCH(high, low, close, k_window, d_window)
    return np.where(slowk > slowd, 1, -1)

# 价格动量
def price_momentum(close, lookback=20):
    ret = close.pct_change(lookback)
    return np.sign(ret)

In [6]:
# 量价突破
def volume_breakout(close, volume, window=20, multiplier=2):
    vol_ma = volume.rolling(window).mean()
    close_ma = close.rolling(window).mean()
    signal = (close > close_ma) & (volume > multiplier * vol_ma)
    return signal.astype(int)

# OBV能量潮
def obv_strategy(close, volume):
    obv = talib.OBV(close, volume)
    return np.where(obv > obv.rolling(20).mean(), 1, -1)

# 成交量加权MACD
def volume_weighted_macd(close, volume, fast=12, slow=26):
    vwma_fast = (close * volume).rolling(fast).sum() / volume.rolling(fast).sum()
    vwma_slow = (close * volume).rolling(slow).sum() / volume.rolling(slow).sum()
    return np.where(vwma_fast > vwma_slow, 1, -1)

In [7]:
# ATR通道突破
def atr_breakout(high, low, close, window=14, multiplier=2):
    atr = talib.ATR(high, low, close, window)
    upper = close.rolling(window).mean() + multiplier * atr
    lower = close.rolling(window).mean() - multiplier * atr
    return np.where(close > upper, 1, np.where(close < lower, -1, 0))

# 布林带收缩
def bollinger_squeeze(close, window=20, std_dev=2):
    ma = close.rolling(window).mean()
    std = close.rolling(window).std()
    bandwidth = (ma + std_dev*std) - (ma - std_dev*std)
    return (bandwidth / ma).rank(pct=True) < 0.2  # 识别波动率极低时期

# 波动率聚类策略
def volatility_regime(close, short_window=10, long_window=50):
    short_vol = close.pct_change().rolling(short_window).std()
    long_vol = close.pct_change().rolling(long_window).std()
    return (short_vol > long_vol).astype(int)  # 高波动 regime=1

In [8]:
# 三重EMA
def triple_ema(close, short=5, medium=20, long=50):
    ema1 = talib.EMA(close, short)
    ema2 = talib.EMA(close, medium)
    ema3 = talib.EMA(close, long)
    return ((ema1 > ema2) & (ema2 > ema3)).astype(int)

# ADX趋势强度
def adx_trend(high, low, close, window=14, threshold=25):
    adx = talib.ADX(high, low, close, window)
    return (adx > threshold).astype(int)

# 抛物线SAR
def sar_strategy(high, low, acceleration=0.02, maximum=0.2):
    sar = talib.SAR(high, low, acceleration, maximum)
    return (close > sar).astype(int)

In [9]:
# 多形态组合
def multi_pattern(open_p, high_p, low_p, close_p):
    patterns = {
        'hammer': talib.CDLHAMMER(open_p, high_p, low_p, close_p),
        'engulfing': talib.CDLENGULFING(open_p, high_p, low_p, close_p),
        'doji': talib.CDLDOJI(open_p, high_p, low_p, close_p)
    }
    signals = pd.DataFrame(patterns)
    return signals.idxmax(axis=1)  # 返回最强形态

In [10]:
def generate_all_signals(ohlcv_data):
    signals = pd.DataFrame(index=ohlcv_data.index)
    
    # 动量类
    signals['rsi'] = rsi_momentum(ohlcv_data['close'])
    signals['macd'] = macd_momentum(ohlcv_data['close'])
    signals['cci'] = cci_momentum(ohlcv_data['high'], ohlcv_data['low'], ohlcv_data['close'])
    
    # 成交量类
    signals['vol_break'] = volume_breakout(ohlcv_data['close'], ohlcv_data['volume'])
    signals['obv'] = obv_strategy(ohlcv_data['close'], ohlcv_data['volume'])
    
    # 波动率类
    signals['atr_break'] = atr_breakout(ohlcv_data['high'], ohlcv_data['low'], ohlcv_data['close'])
    
    # 趋势类
    signals['triple_ema'] = triple_ema(ohlcv_data['close'])
    
    # K线形态
    signals['candle_pattern'] = multi_pattern(
        ohlcv_data['open'], ohlcv_data['high'],
        ohlcv_data['low'], ohlcv_data['close'])
    
    return signals

In [11]:
signals_bit=generate_all_signals(df)
signals_bit

,rsi,macd,cci,vol_break,obv,atr_break,triple_ema,candle_pattern
0,0,-1,0,0,-1,0,0,hammer
1,0,-1,0,0,-1,0,0,hammer
2,0,-1,0,0,-1,0,0,hammer
3,0,-1,0,0,-1,0,0,hammer
4,0,-1,0,0,-1,0,0,hammer
...,...,...,...,...,...,...,...,...
525995,1,-1,1,0,-1,-1,0,hammer
525996,0,-1,1,0,-1,-1,0,hammer
525997,0,-1,0,0,-1,-1,0,doji
525998,1,-1,0,0,-1,-1,0,hammer


In [12]:
def create_ml_features(signals, price_data):
    features = signals.copy()
    
    # 添加交互特征
    features['momentum_ensemble'] = signals[['rsi','macd','cci']].mean(axis=1)
    features['vol_trend_interaction'] = signals['vol_break'] * signals['triple_ema']
    
    # 添加统计特征
    for col in signals.columns:
        features[f'{col}_zscore'] = (signals[col] - signals[col].rolling(30).mean()) / signals[col].rolling(30).std()
    
    # 添加滞后特征
    for lag in [1, 3, 5]:
        for col in ['momentum_ensemble', 'atr_break']:
            features[f'{col}_lag{lag}'] = features[col].shift(lag)
    
    return features.dropna()

In [13]:
# 数据准备
signals = generate_all_signals(df)
features = create_ml_features(signals, df['close'])
labels = (df['close'].shift(-5) > df['close']).astype(int).dropna()

# 构建Pipeline
model = Pipeline([
    ('scaler', StandardScaler()),
    ('xgb', XGBClassifier(
        n_estimators=100,
        max_depth=3,
        learning_rate=0.1,
        subsample=0.8
    ))
])

# 训练与评估
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2)
model.fit(X_train, y_train)
print(classification_report(y_test, model.predict(X_test)))

DataError: No numeric types to aggregate

In [10]:
lookbacks = [5, 7, 10, 14, 21]
percentiles = [90, 95, 97]  
vol_filter = True  

results_rsi = []

def preprocess_data(df):
    # remove the outliers
    df = df[(df['close'].pct_change().abs() < 0.05)]  # filter 5% change
    
    # dealing with the missing values
    df['volume'].replace(0, np.nan, inplace=True)
    df['volume'] = df['volume'].ffill()
    
    # adding volatility filter
    df['atr'] = AverageTrueRange(high=df['high'], low=df['low'], close=df['close'], window=14).average_true_range()
    return df

def generate_signals(df, lookback, percentile):
    """信号生成优化"""
    # RSI calculation
    df['rsi'] = RSIIndicator(close=df['close'], window=lookback).rsi()
    
    # momentum improvement: using 3-period difference and normalizing
    df['rsi_momentum'] = df['rsi'].diff(3)
    df['rsi_momentum'] = (df['rsi_momentum'] - df['rsi_momentum'].rolling(24*60).mean()) / df['rsi_momentum'].rolling(24*60).std()
    
    # dynamic threshold calculation(rolling-window quantile)
    threshold = df['rsi_momentum'].abs().rolling(window=6*60).quantile(percentile/100)
    
    # signal generation
    cond_long = (df['rsi_momentum'] > threshold) & \
               (df['rsi_momentum'].shift(1) > threshold.shift(1))  # continuous 2-period upward trend
    cond_short = (df['rsi_momentum'] < -threshold) & \
                (df['rsi_momentum'].shift(1) < -threshold.shift(1))
    
    df['signal'] = 0
    df.loc[cond_long, 'signal'] = 1
    df.loc[cond_short, 'signal'] = -1
    
    # volatility filter
    if vol_filter:
        df['signal'] = df['signal'] * (df['atr'] < df['atr'].rolling(24*60).quantile(0.75)).astype(int)
    
    return df

def calculate_returns(df):
    # 
    df['return'] = df['close'].pct_change().shift(-1)
    
    # 止损逻辑（2倍ATR跟踪止损）
    df['long_stop'] = (df['close'] - 2*df['atr']).where(df['signal'] == 1).ffill()
    df['short_stop'] = (df['close'] + 2*df['atr']).where(df['signal'] == -1).ffill()
    
    # 生成有效信号
    df['position'] = df['signal']
    df.loc[(df['close'] < df['long_stop']) & (df['signal'] == 1), 'position'] = 0
    df.loc[(df['close'] > df['short_stop']) & (df['signal'] == -1), 'position'] = 0
    
    df['strategy_return'] = df['position'] * df['return']
    return df

for symbol in symbols:
    df = pd.read_csv(f"{data_path}{symbol}_1min_2024-05-01_to_2025-05-01.csv", parse_dates=["timestamp"])
    df.set_index("timestamp", inplace=True)
    df = df.sort_index().pipe(preprocess_data)
    
    for lookback in lookbacks:
        for percentile in percentiles:  # 参数网格扩展
            df_temp = df.copy().pipe(generate_signals, lookback, percentile).pipe(calculate_returns)
            
            # 排除无信号时段
            valid_signals = df_temp[df_temp['position'] != 0]
            
            if not valid_signals.empty:
                hit_rate = (np.sign(valid_signals['position']) == np.sign(valid_signals['strategy_return'])).mean()
                sharpe = valid_signals['strategy_return'].mean() / valid_signals['strategy_return'].std()
                profit_factor = valid_signals[valid_signals['strategy_return'] > 0]['strategy_return'].sum() / \
                              abs(valid_signals[valid_signals['strategy_return'] < 0]['strategy_return'].sum())
            else:
                hit_rate = sharpe = profit_factor = 0
                
            results_rsi.append({
                "symbol": symbol,
                "lookback": lookback,
                "percentile": percentile,
                "hit_rate": round(hit_rate, 3),
                "sharpe_ratio": round(sharpe, 2),
                "profit_factor": round(profit_factor, 2),
                "signals": len(valid_signals),
                "avg_holding_bars": valid_signals['position'].diff().ne(0).cumsum().value_counts().mean()  # 平均持仓时间
            })

results_rsi_df = pd.DataFrame(results_rsi)
results_rsi_df

,symbol,lookback,percentile,hit_rate,sharpe_ratio,profit_factor,signals,avg_holding_bars
0,BTCUSDT,5,90,0.465,-0.00,0.99,12467,2.208503
1,BTCUSDT,5,95,0.464,-0.01,0.97,5226,2.259403
2,BTCUSDT,5,97,0.457,-0.05,0.86,2812,2.229976
3,BTCUSDT,7,90,0.473,-0.01,0.97,13111,2.278193
4,BTCUSDT,7,95,0.467,-0.02,0.95,5514,2.301336
5,BTCUSDT,7,97,0.463,-0.02,0.93,3000,2.295333
6,BTCUSDT,10,90,0.476,-0.00,0.99,13476,2.323448
7,BTCUSDT,10,95,0.472,-0.00,0.98,5804,2.352655
8,BTCUSDT,10,97,0.463,-0.00,0.99,3172,2.311953
9,BTCUSDT,14,90,0.478,0.00,1.00,13804,2.405297


In [11]:
lookbacks = [5, 7, 10, 14, 21]
percentile = 95

# Store all results
results_rsi = []

# Loop over symbols and lookback windows
for symbol in symbols:
    df = pd.read_csv(f"{data_path}{symbol}_1min_2024-05-01_to_2025-05-01.csv", parse_dates=["timestamp"])
    df.set_index("timestamp", inplace=True)
    df = df.sort_index()

    for lookback in lookbacks:
        df_temp = df.copy()
        df_temp["rsi"] = RSIIndicator(close=df_temp["close"], window=lookback).rsi()
        df_temp["rsi_momentum"] = df_temp["rsi"].diff()

        threshold = np.percentile(df_temp["rsi_momentum"].abs().dropna(), percentile)

        df_temp["signal"] = 0
        df_temp.loc[df_temp["rsi_momentum"] > threshold, "signal"] = 1
        df_temp.loc[df_temp["rsi_momentum"] < -threshold, "signal"] = -1

        df_temp["return"] = df_temp["close"].pct_change().shift(-1)
        df_temp["strategy_return"] = df_temp["signal"] * df_temp["return"]

        hit_rate = (np.sign(df_temp["signal"]) == np.sign(df_temp["return"])).mean()
        sharpe = df_temp["strategy_return"].mean() / df_temp["strategy_return"].std()
        total_signals = df_temp["signal"].abs().sum()

        results_rsi.append({
            "symbol": symbol,
            "lookback": lookback,
            "threshold": round(threshold, 2),
            "hit_rate": round(hit_rate, 3),
            "sharpe_ratio": round(sharpe, 2),
            "signals": int(total_signals)
        })

# Present results as a DataFrame
results_rsi_df = pd.DataFrame(results_rsi)
results_rsi_df

,symbol,lookback,threshold,hit_rate,sharpe_ratio,signals
0,BTCUSDT,5,25.18,0.050,0.0,26300
1,BTCUSDT,7,18.15,0.050,0.0,26300
2,BTCUSDT,10,12.85,0.050,0.0,26300
3,BTCUSDT,14,9.27,0.049,0.0,26300
4,BTCUSDT,21,6.25,0.049,0.0,26299


### 2. MACD

In [12]:
results_macd = []

for symbol in symbols:
    df = pd.read_csv(f"{data_path}{symbol}_1min_2024-05-01_to_2025-05-01.csv", parse_dates=["timestamp"])
    df.set_index("timestamp", inplace=True)
    df = df.sort_index()

    # Calculate MACD and MACD Signal Line
    macd = MACD(close=df["close"], window_slow=26, window_fast=12, window_sign=9)
    df["macd"] = macd.macd()
    df["macd_signal"] = macd.macd_signal()
    df["macd_diff"] = df["macd"] - df["macd_signal"]

    # Generate signal based on MACD crossover
    df["signal"] = 0
    df.loc[(df["macd_diff"] > 0) & (df["macd_diff"].shift(1) <= 0), "signal"] = 1
    df.loc[(df["macd_diff"] < 0) & (df["macd_diff"].shift(1) >= 0), "signal"] = -1

    # Simulate returns
    df["return"] = df["close"].pct_change().shift(-1)
    df["strategy_return"] = df["signal"] * df["return"]

    # Evaluate performance
    hit_rate = (np.sign(df["signal"]) == np.sign(df["return"])).mean()
    sharpe = df["strategy_return"].mean() / df["strategy_return"].std()
    total_signals = df["signal"].abs().sum()

    results_macd.append({
        "symbol": symbol,
        "hit_rate": round(hit_rate, 3),
        "sharpe_ratio": round(sharpe, 2),
        "signals": int(total_signals)
    })

# Present results
results_macd_df = pd.DataFrame(results_macd)
results_macd_df

,symbol,hit_rate,sharpe_ratio,signals
0,BTCUSDT,0.064,0.01,42306


### 3. Stochastic Oscillator

In [13]:
results_stoch = []

for symbol in symbols:
    df = pd.read_csv(f"{data_path}{symbol}_1min_2024-05-01_to_2025-05-01.csv", parse_dates=["timestamp"])
    df.set_index("timestamp", inplace=True)
    df = df.sort_index()

    # Calculate Stochastic Oscillator
    stoch = StochasticOscillator(high=df["high"], low=df["low"], close=df["close"], window=14, smooth_window=3)
    df["stoch_k"] = stoch.stoch()
    df["stoch_d"] = stoch.stoch_signal()

    # Generate signal: Buy when %K crosses above %D from oversold, Sell when %K crosses below %D from overbought
    df["signal"] = 0
    df.loc[(df["stoch_k"] < 20) & (df["stoch_k"] > df["stoch_d"]) & (df["stoch_k"].shift(1) <= df["stoch_d"].shift(1)), "signal"] = 1
    df.loc[(df["stoch_k"] > 80) & (df["stoch_k"] < df["stoch_d"]) & (df["stoch_k"].shift(1) >= df["stoch_d"].shift(1)), "signal"] = -1

    # Simulate returns
    df["return"] = df["close"].pct_change().shift(-1)
    df["strategy_return"] = df["signal"] * df["return"]

    # Evaluate
    hit_rate = (np.sign(df["signal"]) == np.sign(df["return"])).mean()
    sharpe = df["strategy_return"].mean() / df["strategy_return"].std()
    total_signals = df["signal"].abs().sum()

    results_stoch.append({
        "symbol": symbol,
        "hit_rate": round(hit_rate, 3),
        "sharpe_ratio": round(sharpe, 2),
        "signals": int(total_signals)
    })

# Display results
results_stoch_df = pd.DataFrame(results_stoch)
results_stoch_df

,symbol,hit_rate,sharpe_ratio,signals
0,BTCUSDT,0.069,0.0,47545


### 4.CCI

In [14]:
results_cci = []

for symbol in symbols:
    df = pd.read_csv(f"{data_path}{symbol}_1min_2024-05-01_to_2025-05-01.csv", parse_dates=["timestamp"])
    df.set_index("timestamp", inplace=True)
    df = df.sort_index()

    # Calculate Commodity Channel Index (CCI)
    cci = CCIIndicator(high=df["high"], low=df["low"], close=df["close"], window=20)
    df["cci"] = cci.cci()

    # Generate signals based on classic CCI strategy
    # Buy when CCI crosses above -100; Sell when CCI crosses below +100
    df["signal"] = 0
    df.loc[(df["cci"] > -100) & (df["cci"].shift(1) <= -100), "signal"] = 1
    df.loc[(df["cci"] < 100) & (df["cci"].shift(1) >= 100), "signal"] = -1

    # Simulate returns
    df["return"] = df["close"].pct_change().shift(-1)
    df["strategy_return"] = df["signal"] * df["return"]

    # Evaluate
    hit_rate = (np.sign(df["signal"]) == np.sign(df["return"])).mean()
    sharpe = df["strategy_return"].mean() / df["strategy_return"].std()
    total_signals = df["signal"].abs().sum()

    results_cci.append({
        "symbol": symbol,
        "hit_rate": round(hit_rate, 3),
        "sharpe_ratio": round(sharpe, 2),
        "signals": int(total_signals)
    })

# Display results
results_cci_df = pd.DataFrame(results_cci)
results_cci_df

,symbol,hit_rate,sharpe_ratio,signals
0,BTCUSDT,0.07,0.0,49523


### 5. Momentum

In [9]:
results_momentum = []

for symbol in symbols:
    df = pd.read_csv(f"{data_path}{symbol}_3min_7days.csv", parse_dates=["timestamp"])
    df.set_index("timestamp", inplace=True)
    df = df.sort_index()

    # Compute Momentum (Rate of Change)
    roc = ROCIndicator(close=df["close"], window=20)
    df["momentum"] = roc.roc()

    # Generate signal
    df["signal"] = 0
    df.loc[df["momentum"] > 0.2, "signal"] = 1
    df.loc[df["momentum"] < -0.2, "signal"] = -1

    # Simulate returns
    df["return"] = df["close"].pct_change().shift(-1)
    df["strategy_return"] = df["signal"] * df["return"]

    # Evaluate
    hit_rate = (np.sign(df["signal"]) == np.sign(df["return"])).mean()
    sharpe = df["strategy_return"].mean() / df["strategy_return"].std()
    total_signals = df["signal"].abs().sum()

    results_momentum.append({
        "symbol": symbol,
        "hit_rate": round(hit_rate, 3),
        "sharpe_ratio": round(sharpe, 2),
        "signals": int(total_signals)
    })

# Display results
results_momentum_df = pd.DataFrame(results_momentum)
results_momentum_df

,symbol,hit_rate,sharpe_ratio,signals
0,BTCUSDT,0.26,0.02,1739


In [ ]:

class BaseSignalStrategy(bt.Strategy):
    params = (('record_signals', True),)

    def __init__(self):
        self.signal = 0  # 当前信号
        self.signals = []  # 记录历史信号

    def next(self):
        if self.record_signals:
            self.signals.append({
                'date': self.data.datetime.date(0),
                'signal': self.signal
            })

    def get_signals_df(self):
        return pd.DataFrame(self.signals).set_index('date')



In [ ]:

# --------------------------
# 独立指标策略（按需扩展）
# --------------------------
class RSIStrategy(BaseSignalStrategy):
    params = (('rsi_period', 14), ('oversold', 30), ('overbought', 70))

    def __init__(self):
        super().__init__()
        self.rsi = bt.indicators.RSI(self.data.close, period=self.p.rsi_period)

    def next(self):
        if self.rsi < self.p.oversold:
            self.signal = 1  # 买入信号
        elif self.rsi > self.p.overbought:
            self.signal = -1  # 卖出信号
        else:
            self.signal = 0
        super().next()



In [ ]:

class CCIStrategy(BaseSignalStrategy):
    params = (('cci_period', 20), ('oversold', -100), ('overbought', 100))

    def __init__(self):
        super().__init__()
        self.cci = bt.indicators.CCI(self.data, period=self.p.cci_period)

    def next(self):
        if self.cci < self.p.oversold:
            self.signal = 1
        elif self.cci > self.p.overbought:
            self.signal = -1
        else:
            self.signal = 0
        super().next()


In [ ]:


class MACDStrategy(BaseSignalStrategy):
    params = (('fast', 12), ('slow', 26), ('signal_period', 9))

    def __init__(self):
        super().__init__()
        self.macd = bt.indicators.MACD(
            self.data.close,
            period_me1=self.p.fast,
            period_me2=self.p.slow,
            period_signal=self.p.signal_period
        )
        self.crossover = bt.indicators.CrossOver(self.macd.macd, self.macd.signal)

    def next(self):
        if self.crossover > 0:
            self.signal = 1  # 金叉买入
        elif self.crossover < 0:
            self.signal = -1  # 死叉卖出
        else:
            self.signal = 0
        super().next()



In [ ]:
class StochasticStrategy(bt.Strategy):
    params = (
        ('k_period', 14),    # %K计算周期
        ('d_period', 3),     # %D计算周期（%K的SMA）
        ('overbought', 80),  # 超买阈值
        ('oversold', 20),    # 超卖阈值
        ('use_trend_filter', True),  # 是否使用趋势过滤
    )

    def __init__(self):
        # 计算指标
        self.stoch = bt.indicators.Stochastic(
            self.data,
            period=self.p.k_period,
            period_dfast=self.p.d_period,
            upperband=self.p.overbought,
            lowerband=self.p.oversold
        )
        self.crossover = bt.indicators.CrossOver(self.stoch.lines.percK, self.stoch.lines.percD)
        
        # 趋势过滤（可选）
        if self.p.use_trend_filter:
            self.sma200 = bt.indicators.SMA(self.data.close, period=200)

    def next(self):
        # 生成信号（1=买入，-1=卖出，0=无信号）
        signal = 0
        
        # 多头条件
        if (self.stoch.lines.percK[0] < self.p.oversold and 
            self.crossover > 0 and 
            (not self.p.use_trend_filter or self.data.close[0] > self.sma200[0])):
            signal = 1
        
        # 空头条件
        elif (self.stoch.lines.percK[0] > self.p.overbought and 
              self.crossover < 0 and 
              (not self.p.use_trend_filter or self.data.close[0] < self.sma200[0])):
            signal = -1
        
        # 记录信号（用于后续机器学习组合）
        self.signal = signal

In [ ]:

class MOMStrategy(BaseSignalStrategy):
    params = (('mom_period', 10), ('threshold', 0))

    def __init__(self):
        super().__init__()
        self.mom = bt.indicators.Momentum(self.data.close, period=self.p.mom_period)

    def next(self):
        if self.mom > self.p.threshold:
            self.signal = 1  # 动量向上
        elif self.mom < -self.p.threshold:
            self.signal = -1  # 动量向下
        else:
            self.signal = 0
        super().next()